In [17]:
from PIL import Image
import os
import torch
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn.functional as F
from tqdm import tqdm
from torch.nn.utils import spectral_norm

In [18]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA")
else:
    device = torch.device("cpu")
    print("Using CPU")

Using CUDA


In [19]:
subject_data=torch.load(r'C:\Users\hp\Desktop\M.Tech\ML\ML_Project\ML_Project-main\instances_&_dict\data_with_embeddings.pth')

C:\Users\hp\AppData\Local\Temp\ipykernel_17800\663694788.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  subject_data=torch.load(r'C:\Users\hp\Desktop\M.Tech\ML\ML_Proje

In [20]:
class EEGImageDataset():
    def __init__(self, subject_data, subject_no, image_folder, synset_csv, transform=None, eeg_length=440):
        """
        Args:
            subject_data (dict): Dictionary with keys `dataset`, `labels`, `images`, `z`.
            subject_no (int): The subject number to filter data.
            image_folder (str): Path to the main image folder.
            synset_csv (str): Path to the synset CSV file with `images` and `folder` columns.
            transform (callable, optional): Optional transform to be applied on the images.
            eeg_length (int): Fixed length for the EEG data (e.g., 440).
        """
        self.eeg_embeddings = subject_data[subject_no]["z"]  # Precomputed EEG embeddings
        self.labels = subject_data[subject_no]["Labels"]  # Labels for the subject
        self.image_folder = image_folder
        self.synset_df = pd.read_csv(synset_csv)  # Read the synset CSV file
        self.transform = transform
        self.eeg_length = eeg_length  # Length of EEG signals, but we don't crop here

        # Ensure we have the same number of EEG samples and images
        num_eegs = len(self.eeg_embeddings)
        num_images = len(self.synset_df)

        if num_eegs < num_images:
            self.synset_df = self.synset_df.iloc[:num_eegs]  # Truncate images to match EEGs
        elif num_eegs > num_images:
            raise ValueError(f"More EEG samples ({num_eegs}) than images ({num_images})")

    def __len__(self):
        return len(self.eeg_embeddings)

    def __getitem__(self, idx):
        eeg_embedding = self.eeg_embeddings[idx].to(torch.float32)  # Shape: (embedding_size)

        label = self.labels[idx]

        image_filename = self.synset_df.iloc[idx]["images"] + ".JPEG"
        folder_name = self.synset_df.iloc[idx]["folder"]
        image_path = os.path.join(self.image_folder, folder_name, image_filename)

        image = Image.open(image_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return {"eeg": eeg_embedding, "image": image, "label": label}


In [21]:
# Define transformations for the images (optional)
image_transforms = transforms.Compose([
    transforms.ToTensor(),  # Convert PIL image to tensor
])

image_folder_path = r"C:\Users\hp\Desktop\M.Tech\ML\ML_Project\ML_Project-main\Subset_128x128"
synset_csv_path = r"C:\Users\hp\Desktop\M.Tech\ML\ML_Project\ML_Project-main\Subset\synsetIds.csv"

# Define the subject number to train for
subject_no = 1  # Change this to train for a different subject (1, 2, 3, ..., 6)

# Create the dataset for the specific subject
eeg_image_dataset = EEGImageDataset(
    subject_data=subject_data,
    subject_no=subject_no,
    image_folder=image_folder_path,
    synset_csv=synset_csv_path,
    transform=image_transforms
)

# Create a DataLoader for batching
data_loader = DataLoader(eeg_image_dataset, batch_size=16, shuffle=True,drop_last=True)

# Example: Iterate through the DataLoader and pass to GAN
for batch in data_loader:
    # Pass EEG embeddings (batch['eeg']) and images (batch['image']) to the GAN
    eeg_embeddings = batch["eeg"]  # Shape: (batch_size, embedding_size)
    images = batch["image"]  # Shape: (batch_size, 3, 128, 128)
    labels = batch["label"]  # Shape: (batch_size,)

    # Example: Pass embeddings and images to the GAN's generator
    # Assuming generator takes both EEG embeddings (z) and images as input
    # generated_images = generator(eeg_embeddings, images)

    # For debugging purposes, let's print the shapes
    print("EEG Embeddings shape:", eeg_embeddings.shape)
    print("Image shape:", images.shape)
    print("z", labels)
    break

EEG Embeddings shape: torch.Size([16, 128])
Image shape: torch.Size([16, 3, 128, 128])
z tensor([14,  3, 16,  0, 22, 24, 39, 15, 15, 20, 38, 10, 32,  0, 36, 28],
       device='cuda:0')


In [5]:
# ResBlockUp (used in Generator)
class ResBlockUp(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResBlockUp, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.upsample = nn.Upsample(scale_factor=2, mode='nearest')
        self.shortcut = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        shortcut = self.upsample(self.shortcut(x))
        x = self.bn1(x)
        x = F.relu(x)
        x = self.upsample(x)
        x = self.conv1(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.conv2(x)
        return x + shortcut


# ResBlockDown (used in Discriminator)
class ResBlockDown(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResBlockDown, self).__init__()
        self.conv1 = spectral_norm(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))  # Apply spectral norm
        self.conv2 = spectral_norm(nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1))  # Apply spectral norm
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = nn.AvgPool2d(kernel_size=2)
        self.shortcut = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=2)

    def forward(self, x):
        shortcut = self.shortcut(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.conv1(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = self.downsample(x)
        return x + shortcut


# Generator
class Generator(nn.Module):
    def __init__(self, noise_dim=100, class_dim=128):
        super(Generator, self).__init__()
        self.fc = nn.Linear(noise_dim + class_dim, 4 * 4 * 1024)
        self.resblock1 = ResBlockUp(1024, 512)
        self.resblock2 = ResBlockUp(512, 256)
        self.resblock3 = ResBlockUp(256, 128)
        self.resblock4 = ResBlockUp(128, 64)
        self.bn = nn.BatchNorm2d(64)
        self.conv = nn.Conv2d(64, 3, kernel_size=3, padding=1)
        self.tanh = nn.Tanh()

    def forward(self, noise, class_embedding):
        x = torch.cat([noise, class_embedding], dim=1)  # Concatenate noise and class embedding
        x = self.fc(x).view(-1, 1024, 4, 4)  # Reshape to 4×4×1024
        x = self.resblock1(x)
        x = self.resblock2(x)
        x = self.resblock3(x)
        x = self.resblock4(x)
        x = self.bn(x)
        x = F.relu(x)
        x = self.conv(x)
        return self.tanh(x)


# Discriminator
class Discriminator(nn.Module):
    def __init__(self, class_dim=128):
        super(Discriminator, self).__init__()
        # Initial ResBlock layers
        self.resblock1 = ResBlockDown(3, 64)
        self.resblock2 = ResBlockDown(64, 128)
        self.resblock3 = ResBlockDown(128, 256)

        # Embedding layer for class embeddings
        self.embed = nn.Linear(class_dim, 256)  # Project class embedding to match feature map channels

        # Continue ResBlocks after concatenation
        self.resblock4 = ResBlockDown(256 + 256, 512)  # Adjust input to account for concatenation
        self.resblock5 = ResBlockDown(512, 1024)
        self.resblock6 = ResBlockDown(1024, 1024)

        # Final layers
        self.relu = nn.ReLU()
        self.global_sum_pooling = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(1024, 1)

    def forward(self, img, class_embedding):
        # Initial feature extraction
        h = self.resblock1(img)
        h = self.resblock2(h)
        h = self.resblock3(h)

        # Get the spatial dimensions of the feature map
        feature_map_size = h.shape[-2:]  # (height, width)

        # Embed class and reshape to match feature map dimensions
        condition_embed = self.embed(class_embedding).unsqueeze(-1).unsqueeze(-1)
        condition_embed = condition_embed.expand(-1, -1, *feature_map_size)

        # Concatenate class embedding with the feature map
        h = torch.cat([h, condition_embed], dim=1)  # Concatenate along channel dimension

        # Continue feature extraction
        h = self.resblock4(h)
        h = self.resblock5(h)
        h = self.resblock6(h)

        # Final processing
        h = self.relu(h)
        h = self.global_sum_pooling(h).view(h.size(0), -1)  # Flatten
        return self.fc(h)

In [6]:
import torch
import torch.nn as nn
from tqdm import tqdm

def train_gan(generator, discriminator, optimizer_gen, optimizer_disc, subject_data, subject_no, dataloader, noise_dim, num_epochs, lr, device):

    # Loss function
    criterion = nn.BCEWithLogitsLoss()

    for epoch in range(num_epochs):
        generator.train()
        discriminator.train()

        epoch_gen_loss = 0.0
        epoch_disc_loss = 0.0

        with tqdm(dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}") as pbar:
            for batch in pbar:
                real_images = batch["image"].to(device)
                eeg_embeddings = batch["eeg"].to(device)  # Precomputed embeddings ('z')

                # Modify EEG embeddings for the first 500 epochs and next 250 epochs
                if epoch < (num_epochs*2)/3:
                    # Set EEG embeddings to zero for the first 500 epochs
                    eeg_embeddings = torch.zeros_like(eeg_embeddings).to(device)
                elif epoch >=(num_epochs*2)/3:
                    # Use original EEG embeddings for the next 250 epochs (500-750)
                    pass
                else:
                    pass

                # === Train Discriminator ===
                optimizer_disc.zero_grad()
                batch_size = real_images.size(0)
                noise = torch.randn(batch_size, noise_dim).to(device)  # Random noise for the generator

                # Generate fake images using the generator
                fake_images = generator(noise, eeg_embeddings)

                # Real and fake labels
                real_labels = torch.full((batch_size, 1), 0.8).to(device)  # Smooth real labels
                fake_labels = torch.full((batch_size, 1), 0.0).to(device)

                # Discriminator loss (real images vs. fake images)
                real_loss = torch.mean(torch.log(discriminator(real_images, eeg_embeddings) + 1e-8))  # log(D(x))
                fake_loss = torch.mean(torch.log(1 - discriminator(fake_images.detach(), eeg_embeddings) + 1e-8))  # log(1-D(G(z)))
                disc_loss = -(real_loss + fake_loss)
                disc_loss.backward()
                optimizer_disc.step()

                # === Train Generator ===
                optimizer_gen.zero_grad()
                fake_images = generator(noise, eeg_embeddings)  # Generate fake images
                gen_loss = criterion(discriminator(fake_images, eeg_embeddings), real_labels)  # Fool Discriminator
                gen_loss.backward()
                optimizer_gen.step()

                # Update epoch losses
                epoch_gen_loss += gen_loss.item()
                epoch_disc_loss += disc_loss.item()

                # Debugging outputs
                if batch_size > 0 and epoch == 0:  # Log first batch outputs
                    print(f"Real Loss: {real_loss.item():.4f}, Fake Loss: {fake_loss.item():.4f}")
                    print(f"Fake Images Mean: {fake_images.mean().item():.4f}")

                # Update tqdm progress bar
                pbar.set_postfix({
                    "Gen Loss": f"{gen_loss.item():.4f}",
                    "Disc Loss": f"{disc_loss.item():.4f}"
                })

        # Print epoch statistics
        print(f"Epoch [{epoch + 1}/{num_epochs}] | Gen Loss: {epoch_gen_loss / len(dataloader):.4f} | Disc Loss: {epoch_disc_loss / len(dataloader):.4f}")


In [ ]:
# Assign hyperparameters
noise_dim = 100
latent_dim = 128  # Latent vector size from EEGEncoder
num_epochs = 30
batch_size = 16
lr = 0.0002

# Initialize models
generator = Generator(noise_dim=noise_dim, class_dim=latent_dim).to(device)
discriminator = Discriminator(class_dim=latent_dim).to(device)

optimizer_gen = torch.optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_disc = torch.optim.Adam(discriminator.parameters(), lr=lr * 0.5, betas=(0.5, 0.999))

# Load dataset and dataloader
image_folder_path = r"C:\Users\CSE IIT BHILAI\EpochHunters\Subset_128x128\Subset_128x128"
synset_csv_path = r"C:\Users\CSE IIT BHILAI\EpochHunters\synsetIds.csv"

# Define image transformations
image_transforms = transforms.Compose([
    transforms.ToTensor(),  # Convert PIL image to tensor
])

# Load the dataset
eeg_image_dataset = EEGImageDataset(
    subject_data=subject_data,
    subject_no=subject_no,
    image_folder=image_folder_path,
    synset_csv=synset_csv_path,
    transform=image_transforms
)

# Create DataLoader
data_loader = DataLoader(eeg_image_dataset, batch_size=batch_size, shuffle=True)

# Start training
train_gan(generator, discriminator,optimizer_gen, optimizer_disc, subject_data, subject_no, data_loader, noise_dim, num_epochs, lr, device)

Epoch 1/30:   1%|▎                                  | 1/125 [00:00<00:58,  2.11it/s, Gen Loss=0.6833, Disc Loss=1.3777]

Real Loss: 0.6649, Fake Loss: 0.7128
Fake Images Mean: 0.0065


Epoch 1/30:   2%|▌                                  | 2/125 [00:00<00:41,  2.93it/s, Gen Loss=0.6779, Disc Loss=1.3958]

Real Loss: 0.6727, Fake Loss: 0.7232
Fake Images Mean: 0.0056


Epoch 1/30:   2%|▊                                  | 3/125 [00:00<00:34,  3.50it/s, Gen Loss=0.6817, Disc Loss=1.3839]

...

Epoch [18/30] | Gen Loss: 0.6819 | Disc Loss: 1.3836


Epoch 19/30: 100%|████████████████████████████████| 125/125 [00:26<00:00,  4.69it/s, Gen Loss=0.7214, Disc Loss=1.3198]


Epoch [19/30] | Gen Loss: 0.6823 | Disc Loss: 1.3826


Epoch 20/30:  38%|████████████▍                    | 47/125 [00:10<00:16,  4.59it/s, Gen Loss=0.6840, Disc Loss=1.3785]


KeyboardInterrupt: 

In [ ]:
# Assign hyperparameters
noise_dim = 100
latent_dim = 128  # Latent vector size from EEGEncoder
num_epochs = 750
batch_size = 16
lr = 0.0002

optimizer_gen = torch.optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_disc = torch.optim.Adam(discriminator.parameters(), lr=lr * 0.5, betas=(0.5, 0.999))

train_gan(generator, discriminator,optimizer_gen,optimizer_disc,subject_data,subject_no,data_loader,noise_dim,num_epochs,lr,device)

Epoch 1/750:   1%|▎                                 | 1/125 [00:00<00:54,  2.26it/s, Gen Loss=2.1920, Disc Loss=0.8810]

Real Loss: 0.7152, Fake Loss: 0.6879
Fake Images Mean: 0.0018
Real Loss: 0.5266, Fake Loss: 0.3544
Fake Images Mean: 0.0046


Epoch 1/750:   2%|▊                                 | 3/125 [00:01<00:30,  4.06it/s, Gen Loss=1.3699, Disc Loss=0.4518]

Real Loss: 0.3830, Fake Loss: 0.1498
Fake Images Mean: 0.0044
Real Loss: 0.4203, Fake Loss: 0.0314
Fake Images Mean: -0.0080


Epoch 1/750:   4%|█▎                                | 5/125 [00:01<00:25,  4.77it/s, Gen Loss=4.7764, Disc Loss=0.5341]

...

Epoch [748/750] | Gen Loss: 23.4965 | Disc Loss: 0.3251


Epoch 749/750: 100%|█████████████████████████████| 125/125 [00:23<00:00,  5.21it/s, Gen Loss=24.2762, Disc Loss=0.3251]


Epoch [749/750] | Gen Loss: 23.4650 | Disc Loss: 0.3251


Epoch 750/750: 100%|█████████████████████████████| 125/125 [00:24<00:00,  5.19it/s, Gen Loss=23.6716, Disc Loss=0.3251]

Epoch [750/750] | Gen Loss: 23.6544 | Disc Loss: 0.3251


## Saving model

In [ ]:
torch.save(generator.state_dict(), r"C:\Users\CSE IIT BHILAI\Desktop\ML_Project-main\GAN_models\generator-main.pth")
torch.save(discriminator.state_dict(), r"C:\Users\CSE IIT BHILAI\Desktop\ML_Project-main\GAN_models\discriminator-main.pth")
torch.save(optimizer_gen.state_dict(), r"C:\Users\CSE IIT BHILAI\Desktop\ML_Project-main\GAN_models\optimizer_gen.pth")
torch.save(optimizer_disc.state_dict(), r"C:\Users\CSE IIT BHILAI\Desktop\ML_Project-main\GAN_models\optimizer_disc.pth")

## Loading & Using the Model to generate Images

In [ ]:
generator = Generator(noise_dim=100, class_dim=128).to(device)
discriminator = Discriminator(class_dim=128).to(device)

generator.load_state_dict(torch.load(r"C:\Users\CSE IIT BHILAI\Desktop\ML_Project-main\GAN_models\generator-main.pth"))
discriminator.load_state_dict(torch.load(r"C:\Users\CSE IIT BHILAI\Desktop\ML_Project-main\GAN_models\discriminator-main.pth"))

generator.eval()
discriminator.eval()

print("Generator and Discriminator models loaded successfully.")

Generator and Discriminator models loaded successfully.


C:\Users\CSE IIT BHILAI\AppData\Local\Temp\ipykernel_9668\2453012757.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  generator.load_state_dict(torch.load(r"C:\Users\CSE 

In [ ]:
# Load optimizers if you plan to continue training
optimizer_gen = torch.optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_disc = torch.optim.Adam(discriminator.parameters(), lr=lr * 0.5, betas=(0.5, 0.999))

optimizer_gen.load_state_dict(torch.load("/content/drive/MyDrive/sngan-models/optimizer_gen.pth"))
optimizer_disc.load_state_dict(torch.load("/content/drive/MyDrive/sngan-models/optimizer_disc.pth"))

In [ ]:
latent_dim = 128  #
noise_dim = 100   

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

noise = torch.randn(1, noise_dim).to(device)  # Shape: (1, 100)
print("Noise:", noise.shape)

latent_vector = subject_data[1]['z'][0]  # Shape: (1, 128)
latent_vector=latent_vector.reshape(1,-1)

x = torch.cat([noise, latent_vector], dim=1)  # Shape: (1, 100 + 128) = (1, 228)
print("Concatenated tensor x:", x.shape)

Noise: torch.Size([1, 100])
Concatenated tensor x: torch.Size([1, 228])


In [ ]:
from torchvision.utils import save_image

In [ ]:
with torch.no_grad():  
    generated_image = generator(noise, latent_vector)  

generated_image = (generated_image + 1) / 2

save_image(generated_image, "generated_image.png")